<h1>DS Con: Scraping Content</h1>

<h2>Importing libraries</h2>

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import mysql.connector
import csv
import getpass

In [2]:
stopwords = []

with open('/home/de-admin/Documents/Webscraping/Pastebin/stopwords/corpora/stopwords/english','rb') as f:
    reader = csv.reader(f)
    for row in reader:
        stopwords += row

In [3]:
pswd = getpass.getpass('Password:')

Password:········


*If you ever needed to re-create the table...*

try:
    cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
    cursor = cnx.cursor(buffered=True)
    cursor.execute("CREATE TABLE pastebindsconflag (LINK varchar(40), TIME varchar(40),  TEXT varchar(60000))")
except mysql.connector.Error as err:
    print(str(err))
else:
    cursor.close()
    cnx.close()

try:
    cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
    cursor = cnx.cursor(buffered=True)
    cursor.execute("CREATE TABLE pastebindsconlarge (LINK varchar(40), TIME varchar(40), TEXT varchar(35000), TEXT_P varchar(25000))")
except mysql.connector.Error as err:
    print(str(err))
else:
    cursor.close()
    cnx.close()

<h2>Defining the Webscraping Spider</h2>

In [4]:
class MySpider(scrapy.Spider):
    name = "Pastebin"

    custom_settings = {
        'DOWNLOAD_DELAY': 2,
        'RETRY_TIMES' : 3
    }
    
    
    def __init__(self, *args, **kwargs):
        super(MySpider, self).__init__(*args, **kwargs) 
        self.pswd=[kwargs.get('pswd')] 
        self.stopwords=[kwargs.get('stopwords')] 
        
    def start_requests(self):
        
        while 1 == 1:
            try:
                cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
                cursor = cnx.cursor(buffered=True)
                cursor.execute("SELECT * FROM pastebin")
                result = cursor.fetchall()
            except mysql.connector.Error as err:
                print(str(err))
            else:
                cursor.close()
                cnx.close()

            for link in result:
                try:
                    cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
                    cursor = cnx.cursor(buffered=True)
                    query = "DELETE FROM pastebin WHERE LINK = '"+link[0]+"'"
                    cursor.execute(query)
                    cnx.commit()
                except mysql.connector.Error as err:
                    print(str(err))
                else:
                    cursor.close()
                    cnx.close()

                yield scrapy.Request(url="https://pastebin.com"+link[0].encode('ascii','ignore'), callback=self.parse, dont_filter=True, meta={'time':link[1]})

    def parse(self, response):
        targets = ["gsi.gov.uk","spotify","netflix","steam key","minecraft nfa","deezer","crunchyroll","mcnfa","hulu","hma key","fortnite","nordvpn"]
        content_raw = response.xpath('//textarea//text()').extract()[0].encode('ascii','ignore')
        content = ""
        
        a = content_raw.split()
        for x in a:
            if len(x) > 3:
                if x not in stopwords:
                    x += " "        
                    content += x.encode('ascii','ignore')     
                    
        if any(x in content_raw.lower() for x in targets):
            try:
                cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
                cursor = cnx.cursor(buffered=True)
                query = "INSERT INTO pastebindsconflag (LINK, TIME, TEXT) VALUES ('"+response.request.url+"','"+response.meta['time']+"', '"+content_raw.encode('hex')+"')"
                cursor.execute(query)
                cnx.commit()
            except mysql.connector.Error as err:
                print(str(err))
            else:
                cursor.close()
                cnx.close()
        else:
            try:
                cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
                cursor = cnx.cursor(buffered=True)
                query = "INSERT INTO pastebindsconlarge (LINK, TIME, TEXT, TEXT_P) VALUES ('"+response.request.url+"','"+response.meta['time']+"', '"+content_raw.encode('hex')+"', '"+content.encode('hex')+"')"
                cursor.execute(query)
                cnx.commit()
            except mysql.connector.Error as err:
                print(str(err))
            else:
                cursor.close()
                cnx.close()

<h2>Running the Webscraping</h2>

Note, you can't re run the code below in a single session for one reason or another, so you need to restart the kernel between runs.

This code creates a lightweight container for our webspider and then runs it - to be honest understanding this is probably optional unless it breaks.

In [ ]:
process = CrawlerProcess()
process.crawl(MySpider, pswd=pswd, stopwords=stopwords)
process.start()

2018-05-18 12:50:59 [scrapy.utils.log] INFO: Scrapy 1.3.3 started (bot: scrapybot)
2018-05-18 12:50:59 [scrapy.utils.log] INFO: Overridden settings: {}
2018-05-18 12:50:59 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats']
2018-05-18 12:50:59 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.downloadermiddlewares.cookies.CookiesMiddle